# ADMM
- Jaan Tollander de Balsch
- 452056

In [ ]:
using JuMP
using Ipopt
using Random
using Printf
using LinearAlgebra

In [ ]:
## Function to solve a small instance of the deterministic model
function deterministic_model_small()

    ##################### PROBLEM DATA FOR SMALLER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 100                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################
    
    ## We first solve the problem formulation directly without ADMM
    tini  = time()                                      # Start timer
    model = Model(with_optimizer(Ipopt.Optimizer))      # We use Ipopt solver to compare with ADMM   

    ## Variables
    @variable(model, x[i in I] >= 0)                    # Reserved capacity variables
    @variable(model, y[i in I, j in J, s in S] >= 0)    # Demand fulfilment variables
    @variable(model, u[j in J, s in S] >= 0)            # Unfulfilled demand variables

    ## Objective: Minimize the total phase 1 + phase 2 costs over all scenarios
    @objective(model, Min,
          sum(c[i]*x[i] for i in I) +
          sum(p[s]*f[i,j]*y[i,j,s] for s in S, i in I, j in J) + 
          sum(p[s]*q[j]*u[j,s] for s in S, j in J))

    ## Constraints
    @constraint(model, [i in I], x[i] <= b[i])          # Max capacity constraint
    @constraint(model, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost) constraint

    ## Capacity reserve limit constraint for each supplier i in each scenario s
    @constraint(model, [i in I, s in S], sum(y[i,j,s] for j in J) <= x[i])

    ## Demand balance constraint (u[j,s] is unfulfilled demand of client j in scenario s)
    @constraint(model, DemBal[j in J, s in S], sum(y[i,j,s] for i in I) + u[j,s] == d[j,s])

    optimize!(model)                                    # Solve the problem

    status = termination_status(model)                  # Solution status
    println(status)                                     # Print status
    
    ## Stop timer
    tend = time() - tini
   
    ## Return solution vector x, costs c, and solution time tend
    return (c, x, tend)
    
end

In [ ]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the small deterministic instance. 
(c, x, tend) = deterministic_model_small()

xsol = value.(x)                          # Get optimal x values (reserved capacities)
xsol = round.(xsol.data, digits = 2)      # Round to 2 decimals 
fval = dot(c, xsol)                       # Optimal cost of reserved capacities

## Solution time of small deterministic model
println("\nSolution time: ", tend, "\n")

## Print optimal solution 
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nOptimal cost of reserved capacities: ", fval)

In [ ]:
## Function to solve a large instance of the deterministic model
function deterministic_model_large()

    ##################### PROBLEM DATA FOR LARGER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 500                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################
    
    ## We first solve the problem formulation directly without ADMM
    tini  = time()                                      # Start timer
    model = Model(with_optimizer(Ipopt.Optimizer))      # We use Ipopt solver to compare with ADMM   

    ## Variables
    @variable(model, x[i in I] >= 0)                    # Reserved capacity variables
    @variable(model, y[i in I, j in J, s in S] >= 0)    # Demand fulfilment variables
    @variable(model, u[j in J, s in S] >= 0)            # Unfulfilled demand variables

    ## Objective: Minimize the total phase 1 + phase 2 costs over all scenarios
    @objective(model, Min,
          sum(c[i]*x[i] for i in I) +
          sum(p[s]*f[i,j]*y[i,j,s] for s in S, i in I, j in J) + 
          sum(p[s]*q[j]*u[j,s] for s in S, j in J))

    ## Constraints
    @constraint(model, [i in I], x[i] <= b[i])          # Max capacity constraint
    @constraint(model, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost) constraint

    ## Capacity reserve limit constraint for each supplier i in each scenario s
    @constraint(model, [i in I, s in S], sum(y[i,j,s] for j in J) <= x[i])

    ## Demand balance constraint (u[j,s] is unfulfilled demand of client j in scenario s)
    @constraint(model, DemBal[j in J, s in S], sum(y[i,j,s] for i in I) + u[j,s] == d[j,s])

    optimize!(model)                                    # Solve the problem

    status = termination_status(model)                  # Solution status
    println(status)                                     # Print status 
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x, tend)
end

In [ ]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the large deterministic model
(c, x, tend) = deterministic_model_large()

xsol = value.(x)                          # Get optimal x values (reserved capacities)
xsol = round.(xsol.data, digits = 2)      # Round to 2 decimals 
fval = dot(c, xsol)                       # Optimal cost of reserved capacities

## Solution time of the large deterministic model
println("\nSolution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nOptimal cost of reserved capacities: ", fval)

In [ ]:
#######################################################################################
####           NOTE: RESTART THE KERNEL BEFORE TESTING THE ADMM CODE.             #####  
####                 MEMORY MAY BECOME AN ISSUE OTHERWISE.                        ##### 
#######################################################################################
#### ADMM using Ipopt (other available solvers do not support quadratic objective) ####
#######################################################################################
using Distributed               # For parallel computing 
using Random                    # For controlling randomness
using Printf                    # For C-style printf macro
using SharedArrays              # For shared memory access

addprocs(4)                     # Use 4 parallel threads (add 4 new workers)

@everywhere using LinearAlgebra # For norm() function 
@everywhere using JuMP          # NOTE: The macro @everywhere makes the loaded packages available to all threads
@everywhere using Ipopt         # Solver capable of solving quadratic objectives

In [ ]:
## Function to solve a small instance of the deterministic model.
##
## TODO: Fill in the missing code inside the function. Compare your solution 
##       to that of the small deterministic model. These should be almost equal.
##
## Input argument: penalty parameter ρ
function admm_model_small(ρ::Float64, k_max::Int64 = 200)
    
    ##################### PROBLEM DATA FOR SMALLER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 100                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################

    tini = time()                      # Start timer
    
    ## NOTE: ADMM approach starts from here. Complete the missing code parts that
    ##       are requested. Code lines to be completed are marked as follows:
    ##       TODO: Complete the following expression. (For example)

    ## Problem parameters and memory allocation. NOTE: Compare with Exercise 10.1
    x_s  = SharedArray(zeros(nI, nS))  # Store each x vector in (x,y)-steps of each scenario
    v_s  = SharedArray(zeros(nI, nS))  # Store each v vector in v-steps of each scenario
    z    = SharedArray(zeros(nI))      # Store z vectors at each ADNM iteration
    
    ## Main loop with 200 ADMM iterations
    k = 0
    while k < k_max
        k += 1
        ## Loop for solving each (x,y) step separately for each scenario s in S
        ## NOTE: Compare with Exercise 10.1
        ## NOTE: The macro @distributed allocates scenario subproblems to available
        ##       threads. The macro @sync ensures that all scenario subproblems
        ##       are solved before proceeding with the code after the for loop
        @sync @distributed for s in S   
            
            ## Model to solve (x,y)-step of the current scenario subproblem s
            scen_m = Model(with_optimizer(Ipopt.Optimizer)) 
            
            @variable(scen_m, x[i in I] >= 0)                    # Reserved capacity variables
            @variable(scen_m, y[i in I, j in J] >= 0)            # Demand fulfilment variables
            @variable(scen_m, u[j in J] >= 0)                    # Unfulfilled demand variables

            @constraint(scen_m, [i in I], x[i] <= b[i])          # Max capacity constraint
            @constraint(scen_m, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost)
            
            ## Capacity reserve limit constraint for each supplier i
            @constraint(scen_m, [i in I], sum(y[i,j] for j in J) <= x[i])
            
            ## Demand balance constraint (u[j] is unfulfilled demand of client j)
            @constraint(scen_m, [j in J], sum(y[i,j] for i in I) == d[j,s] - u[j])

            #### TODO: Complete this objective to compute (x, y) - step for the
            ####       current scenario. Compare with Exercise 10.1
            @objective(scen_m, Min, 
                sum(c[i]*x[i] for i in I)
                + sum(f[i,j]*y[i,j] for i in I for j in J)
                + sum(q[j]*u[j] for j in J)
                + sum(v_s[i, s]*x[i] for i in I)
                + ρ/2*sum([(x[i] - z[i])^2 for i in I])
            )

            ## Redirect stdout to omit unnecessary output from workers
            tmp = stdout           
            redirect_stdout()

            ## Solve the (x,y) step for the current scenario
            optimize!(scen_m)

            ## Restore stdout back to normal 
            redirect_stdout(tmp)  

            ## Store the value of x for the current scenario
            x_s[:,s] = value.(x[:])
        end

        ## Compute primal and dual residuals. We use SharedArray to exploit parallelism
        tol = SharedArray(zeros(nS))
        
        #### TODO: Complete the computation of the residual for each s inside the for loop.
        ####       Compare with Exercise 10.1
        @sync @distributed for s in S
            tol[s] = p[s]*ρ*norm(x_s[:,s] .- z)
        end
        
        ## Total residual = sum of subproblem residuals
        tol = sum(tol[s] for s in S)

        ## Print current progress
#         println("iteration: ", k ,"/ residual: ", tol)
        
        ## Stopping condition: if primal + dual residual is small enough
        if tol < 1e-1
            ## Stop timer
            tend = time() - tini
            return (c, x_s, tend, k)
        end

        #### TODO: Complete the z-step. Compare with Exercise 10.1
        z = sum(p[s]*x_s[:,s] for s in S)

        #### TODO: Complete the v-steps for each scenario s.
        ####       Compare with exercise 10.1
        @sync @distributed for s in S
            v_s[:,s] = v_s[:,s] .+ ρ*(x_s[:,s] .- z)
        end
    end
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x_s, tend, k)    
end

In [ ]:
# Compile the function
admm_model_small(100.0, 1);  

In [ ]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the small ADMM model

## Try different values of penalty parameter ρ ∈ [1.0, 100.0]
ρ = 1.0

(c, x_s, tend, k) = admm_model_small(ρ)

xsol = round.(x_s[:,1], digits = 2)     # Get optimal x values (reserved capacities)
fval = dot(c, x_s[:,1])                 # Optimal cost of reserved capacities

## Solution time of ADMM:
println("\nADMM Solution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nCost of reserved capacity: ", fval)

In [ ]:
## Function to solve a large instance of the deterministic model.
##
## TODO: Fill in the missing code inside the function. Compare your solution 
##       to that of the large deterministic model. These should be almost equal.
##
## Input argument: penalty parameter ρ
function admm_model_large(ρ::Float64, k_max::Int64 = 200)
    
    ##################### PROBLEM DATA FOR LARGER INSTANCE #########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 500                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################

    tini = time()                      # Start timer
    
    ## NOTE: ADMM approach starts from here. Complete the missing code parts that
    ##       are requested. Code lines to be completed are marked as follows:
    ##       TODO: Complete the following expression. (For example)

    ## Problem parameters and memory allocation. NOTE: Compare with Exercise 10.1
    x_s  = SharedArray(zeros(nI, nS))  # Store each x vector in (x,y)-steps of each scenario
    v_s  = SharedArray(zeros(nI, nS))  # Store each v vector in v-steps of each scenario
    z    = SharedArray(zeros(nI))      # Store z vectors at each ADNM iteration
    
    ## Main loop with 200 ADMM iterations
    k = 0
    while k < k_max
        k += 1
        ## Loop for solving each (x,y) step separately for each scenario s in S
        ## NOTE: Compare with Exercise 10.1
        ## NOTE: The macro @distributed allocates scenario subproblems to available
        ##       threads. The macro @sync ensures that all scenario subproblems
        ##       are solved before proceeding with the code after the for loop
        @sync @distributed for s in S   
            
            ## Model to solve (x,y)-step of the current scenario subproblem s
            scen_m = Model(with_optimizer(Ipopt.Optimizer)) 
            
            @variable(scen_m, x[i in I] >= 0)                    # Reserved capacity variables
            @variable(scen_m, y[i in I, j in J] >= 0)            # Demand fulfilment variables
            @variable(scen_m, u[j in J] >= 0)                    # Unfulfilled demand variables

            @constraint(scen_m, [i in I], x[i] <= b[i])          # Max capacity constraint
            @constraint(scen_m, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost)
            
            ## Capacity reserve limit constraint for each supplier i
            @constraint(scen_m, [i in I], sum(y[i,j] for j in J) <= x[i])
            
            ## Demand balance constraint (u[j] is unfulfilled demand of client j)
            @constraint(scen_m, [j in J], sum(y[i,j] for i in I) == d[j,s] - u[j])

            #### TODO: Complete this objective to compute (x, y) - step for the
            ####       current scenario. Compare with Exercise 10.1
            @objective(scen_m, Min, 
                sum(c[i]*x[i] for i in I)
                + sum(f[i,j]*y[i,j] for i in I for j in J)
                + sum(q[j]*u[j] for j in J)
                + sum(v_s[i, s]*x[i] for i in I)
                + ρ/2*sum([(x[i] - z[i])^2 for i in I])
            )

            ## Redirect stdout to omit unnecessary output from workers
            tmp = stdout           
            redirect_stdout()

            ## Solve the (x,y) step for the current scenario
            optimize!(scen_m)

            ## Restore stdout back to normal 
            redirect_stdout(tmp)  

            ## Store the value of x for the current scenario
            x_s[:,s] = value.(x[:])
        end

        ## Compute primal and dual residuals. We use array to exploit parallelism
        tol = SharedArray(zeros(nS))
        
        #### TODO: Complete the computation of the residual for each s inside the for loop.
        ####       Compare with Exercise 10.1
        @sync @distributed for s in S
            tol[s] = p[s]*ρ*norm(x_s[:,s] .- z)
        end
        
        ## Total residual = sum of subproblem residuals
        tol = sum(tol[s] for s in S)

        ## Print current progress
#         println("iteration: ", k ,"/ residual: ", tol)
        
        ## Stopping condition: if primal + dual residual is small enough
        if tol < 1e-1
            ## Stop timer
            tend = time() - tini
            return (c, x_s, tend, k)
        end

        #### TODO: Complete the z-step. Compare with Exercise 10.1
        z = sum(p[s]*x_s[:,s] for s in S)

        #### TODO: Complete the v-steps for each scenario s.
        ####       Compare with exercise 10.1
        @sync @distributed for s in S
            v_s[:,s] = v_s[:,s] .+ ρ*(x_s[:,s] .- z)
        end
    end
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x_s, tend, k)    
end

In [ ]:
# Compile the function
admm_model_large(100.0, 1);  

In [ ]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the large ADMM model

## Try different values of penalty parameter ρ ∈ [1.0, 100.0]
ρ = 1.0                                   

(c, x_s, tend, k) = admm_model_large(ρ)

xsol = round.(x_s[:,1], digits = 2)     # Get optimal x values (reserved capacities)
fval = dot(c, x_s[:,1])                 # Optimal cost of reserved capacities

## Solution time of ADMM:
println("\nADMM Solution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nCost of reserved capacity: ", fval)

In [ ]:
## Kill threads
rmprocs(procs())

# Plots

Number of iterations $k$ to convergence as a function of the penalty parameter $\rho$.

In [ ]:
using Plots
using LaTeXStrings

## Small Instance

In [ ]:
ks = []
ρs = vcat(1:5, [7, 10, 15], 20:10:100)
for ρ in ρs
    (c, x_s, tend, k) = admm_model_small(float(ρ))
    println("ρ: ", ρ, " k: ", k)
    push!(ks, k)
end

In [ ]:
if !ispath("figures")
    mkdir("figures")
end

In [ ]:
plot(
    ρs, ks, 
    xticks = ρs,
    xlabel = L"\rho", 
    ylabel = L"k", 
    marker = :circle, 
    markersize = 1, 
    label = "Small Instance",
    legend = :right
)

In [ ]:
#savefig("figures/admm_small_instance.svg")

## Large Instance

In [ ]:
ks2 = []
ρs2 = vcat(1:5, [7, 10, 15], 20:10:100)
for ρ in ρs2
    (c, x_s, tend, k) = admm_model_large(float(ρ))
    println("ρ: ", ρ, " k: ", k)
    push!(ks2, k)
end

In [ ]:
plot(
    ρs2, ks2, 
    xticks = ρs2,
    xlabel = L"\rho", 
    ylabel = L"k", 
    marker = :circle, 
    markersize = 1, 
    label = "Large Instance",
    legend = :right
)

In [ ]:
#savefig("figures/admm_large_instance.svg")